In [6]:
from nanodo.model_factory import *
from nanodo.data import *
from nanodo.configs.default import *
from nanodo.train import *
import numpy as np
import orbax.checkpoint as ocp
from orbax.checkpoint import PyTreeCheckpointer
from nanodo import data_custom
from flax.core import unfreeze
import jax.numpy as jnp
from nanodo.model import *
from flax.linen import Partitioned

In [2]:
c = get_config()
#checkpoint = "/home/allanz/nanodo_workdir/92000.orbax-checkpoint-tmp-138"
checkpoint = "/home/allanz/nanodo_workdir/86000/state"
params= PyTreeCheckpointer().restore(checkpoint)
params = params['params']

/home/allanz/miniconda3/envs/nanodo/lib/python3.11/site-packages/orbax/checkpoint/_src/serialization/type_handlers.py:1250: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [3]:
test_config = ml_collections.config_dict.create(
      D=512,  # model/embed dim  = qkv dim
      F=2048,  # FF inner dimension
      H=8,  # num attention heads
      L=128,  # max context/sequence length (move out of config?)
      N=6,  # number of transformer block layers
      dtype="float32",  # computation dtype.
      fsdp_enabled=True,  # True to shard the model.
      remat=False,  # Transformer block gradient checkpointing to save memory.
  )


In [4]:
tokenizer = get_py_tokenizer("tests/testdata/sentencepiece_cc_all.32000.100extra-sentencepiece.model")
vocab_size = tokenizer.GetPieceSize()
cfg = DoConfig(**test_config, V=vocab_size)  # pytype:disable=attribute-error
# model without float32
float32 = TransformerDo(cfg) 
print(float32)
# model with bfloat16
bfloat16, _ = get_model_and_loss(c, vocab_size)
print(bfloat16)
# model with jax.numpy.float32
jax_float32 = model.DoConfig(D=512, H=8, L=128, N=6, V=vocab_size, F=2048)
m = model.TransformerDo(jax_float32)
print(m)


TransformerDo(
    # attributes
    docfg = DoConfig(D=512, H=8, L=128, N=6, V=32101, F=2048, kernel_init=<function variance_scaling.<locals>.init at 0x7f1f467d87c0>, embed_init=<function variance_scaling.<locals>.init at 0x7f1f467d8900>, dtype='float32', fsdp_enabled=True, remat=False)
)
TransformerDo(
    # attributes
    docfg = DoConfig(D=512, H=8, L=128, N=6, V=32101, F=2048, kernel_init=<function variance_scaling.<locals>.init at 0x7f1f467d87c0>, embed_init=<function variance_scaling.<locals>.init at 0x7f1f467d8900>, dtype='bfloat16', fsdp_enabled=True, remat=False)
)
TransformerDo(
    # attributes
    docfg = DoConfig(D=512, H=8, L=128, N=6, V=32101, F=2048, kernel_init=<function variance_scaling.<locals>.init at 0x7f1f467d87c0>, embed_init=<function variance_scaling.<locals>.init at 0x7f1f467d8900>, dtype=<class 'jax.numpy.float32'>, fsdp_enabled=True, remat=False)
)


In [47]:
train_set = py_batched_tfds(
          tfds_name="c4",
          split="train",
          context_size=128,
          worker_count=0,
          vocab_path="tests/testdata/sentencepiece_cc_all.32000.100extra-sentencepiece.model",
          batch_size = 8
          )
batch = next(iter(train_set))

tf_set = data_custom.py_batched_tfds(
          tfds_name="lm1b",
          split="train",
          context_size=128,
          worker_count=0,
          vocab_path="tests/testdata/sentencepiece_cc_all.32000.100extra-sentencepiece.model",
          batch_size = 8
          )
test_batch = next(iter(tf_set))

ERROR:absl:Applying deprecated PyGrain MapOperation. Please use the grain.python.MapTransform.
ERROR:absl:Applying deprecated PyGrain MapOperation. Please use the grain.python.MapTransform.


In [7]:
def make_partitioned(array, names):
    partition_array = Partitioned(array, names = names, mesh = None)
    return partition_array

def convert_attn_blocks(params):
    blocks = ["blocks_0", "blocks_1", "blocks_2", "blocks_3", "blocks_4", "blocks_5"]
    switches = {"attn_out_proj": (None, None, 'data'), "key": ('data', None), "query":('data', None), "value":('data', None)}

    for block in blocks:
          for switch in switches: 
               #print(params[block]["CausalAttn_0"][switch]["kernel"])
               params[block]["CausalAttn_0"][switch]["kernel"] = make_partitioned(params[block]["CausalAttn_0"][switch]["kernel"]["value"], switches[switch])

def convert_Mlp(params):
    blocks = ["blocks_0", "blocks_1", "blocks_2", "blocks_3", "blocks_4", "blocks_5"]
    switches = {"Dense_0": ('data', None), "Dense_1": ('data', None)}

    for block in blocks:
          for switch in switches: 
               #print(params[block]["Mlp_0"][switch]["kernel"])
               params[block]["Mlp_0"][switch]["kernel"] = make_partitioned(params[block]["Mlp_0"][switch]["kernel"]["value"], switches[switch])


def convert_embed(params):
    params["embed"]["embedding"] = make_partitioned(params["embed"]["embedding"]["value"], (None, 'data'))

def convert_pos_embed(params):
    params["pos_embed"]["embedding"] = make_partitioned(params["pos_embed"]["embedding"]["value"], (None, 'data'))


In [8]:
convert_attn_blocks(params)
convert_Mlp(params)
convert_embed(params)
convert_pos_embed(params)

In [48]:
print(batch, batch.shape, '\n')
print(test_batch, test_batch.shape)

[[    2    97   635 ...  7718 16837    92]
 [ 9425  6921 18822 ...   707     6   142]
 [    4    10   311 ...   249  2273    12]
 ...
 [ 2764  4189    13 ...    10  1026   775]
 [    6    97   197 ...    38   512   775]
 [   14     9   940 ...     8  5769    12]] (8, 128) 

[[    2   217   244 ...  3630    13  1823]
 [  662    10  6416 ...  1535    16 10205]
 [ 3962     4     7 ...   168  3363  1439]
 ...
 [  176   379     4 ...    19  7394    19]
 [ 1491   109  7429 ...  6218 11613    27]
 [    4     6     1 ...  2858  1105    31]] (8, 128)


In [50]:
logits = float32.apply({"params":params}, batch)
print(logits.shape)

(8, 128, 32101)


In [49]:
# input
for x in batch:
    print(tokenizer.decode_ids(x.tolist()), "\n")


"The Perfect Personality For Your Special Occasion" DJ Diana M, Presenter and Event Hostess, is the ideal personality for the more discerning and sophisticated venue seeking someone who is more than just a mere DJ. Woth her huge selection of music covering most genres from the last 80 years she has something for everyone and for any occasion! Required by many venues, DJ Diana DJ has a current PLI policy with AMP DJ in the amount of £10 million. Also required by many venues, DJ Diana DJ has not provided satisfactory evidence of having current PAT records available. A Method Statement out 

lining DJ Diana DJ&apos;s Health & Safety procedure for setting up disco equipment has been provided. Why do people choose to book DJ Diana DJ? I must admit the price was probably one of the first deciding factors and after speaking briefly to them, I have a good feeling about booking them. sensible price, but more importantly very professional and straight forward to deal with over the phone, and by 

In [51]:
# output of the model??? 
probs = jax.nn.softmax(logits, axis=-1)
token_ids = jnp.argmax(probs, axis=-1) 

for x in token_ids:
    print(tokenizer.decode_ids(x.tolist()), "\n")

TheI  Placeity" You Personal Needpinionsion"s . ing of DJ Managersss,   first venue for  event thandiscerning and  DJ. to who is   just a  . Sheon musicly of DJ,  ofs, classic classic decades, has  for everyone.  all occasion. for the of, the Diana M Diana a wide record& and for a ands the UK of £0,00., is the venues, DJ Diana DJ has  only any service of the  PLIs.. DJ for of  

,s and,apos;s  and& Fitness.. the up ss. been developed by The is I have to use & DJ& DJs say, fact of very the of the most things& factors in I all with about the about I was toa feeling feeling that the DJ. I and. but I importantly, good service friendly forward. the with the  phone. and they the. Ien to get you best of service that need. IIcellent service professional Diana DJ  of instruments and needed and . was for. hours. I wanted like to have. my CD. He  

a  nice idea’ you. III you for thank was  to have  fors . and and, are the night. well. "I ared the  well well and wered the. needed to for. theyliste

In [29]:
# input
for x in test_batch:
    print(tokenizer.decode_ids(x.tolist()), "\n")


He said Epperson told Edgewood High School administrators that the students -- all cheerleaders at the school north of Cincinnati in Trenton -- asked her to take them to the bar in February . The Fighting Irish rank 110th among the 120 major college teams . Brilliance Auto was the first Chinese automaker to set up shop in the United Kingdom , initially offering the midsize BS6 sedan and most recently adding the compact BS4 . The charity has called on the government 's Missing Persons Taskforce to ensure its forthcoming action plan includes effective measures to protect 

runaways from pimps . That 's the message U.S. Education Secretary Arne Duncan delivered to a California audience this summer . It is the telephone and obligations that make people tense . During the Olympic team 's practice at Stanford on Saturday , the rhythmic slap , slap , slap of swimmers slicing through the water was mixed with peals of laughter . In 1956 , he moved to St. Louis , where he met Anna Mae Bull 

ock

In [30]:
# output of the model??? 
logits = float32.apply({"params":params}, test_batch)
print(logits.shape)
probs = jax.nn.softmax(logits, axis=-1)
token_ids = jnp.argmax(probs, axis=-1) 

for x in token_ids:
    print(tokenizer.decode_ids(x.tolist()), "\n")

(8, 128, 32101)
The was thathe was thewater that School that that  school were who ofleaders -- the school -- of the -- theon -- were the to be the to the school. the.if " The  Irishman inth in the topth league football inranked The Theilliance is Repair founded first of manufacturermaker to be up  in China country States.. and in  - sedan,- s the recently the the  BS6 sedanX The The  is been for the government toas ing's' Force' be that independence  on is  measures to ensure the 

.ways. theggying and&  The’as not  I.S. President Secretary,ne Duncan has to thea meeting school in week.-  The’  first number the of are us aam and.  The ⁇  the last Games,,The , the University the,, the team of tempoas. thealap , slap  thes ,lapked  the    with thee   , The The the,, thea was to the. Louis,, Missouri he was his ande anda 

s&  I have ,signed the a  ,  TheI'ss   the to to- but I. said me. John Nixon, the that, who state- in the floor., " The and in the that  was thethe controle down the  '

In [32]:
from datasets import load_dataset

testing = load_dataset("stas/c4-en-10k")